<a href="https://colab.research.google.com/github/MOHOAzure/AI_Paint_Online/blob/main/Colab/WebUI_OpenSrc_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Painting tool
* https://github.com/AUTOMATIC1111/stable-diffusion-webui

# Source of open source models
* https://huggingface.co/
* https://civitai.com/

# Munual

1. In Configuration, setup `account & pwd` to login to the Painting Tool. In order to prevent others from crawling your online deployment for use, it is strongly recommended to setup the account and password.

2. (Optional) In Configuration, pick up features you need. E,g.,
  * Models
  * CUDA Calculation
  * SuperMerger
  * ControlNet

3. In the navigation bar, click `Runtime (執行階段) > Run all (全部執行)` (or press: `Ctrl+F9`). Waiting setup to complete, and then you will get something like xxx.gradio.app. Click it to go to the launched Painting tool.

4. After finish painting. In the navigation bar, click `Disconnect > Terminate`



In [1]:
#@title Configuration

from pathlib import Path


# define variables such as working directory
root_dir = Path("/content")
tool_dir = root_dir/"stable-diffusion-webui"

account = "MY_ACCOUNT" #@param {type:"string"}
pwd = "MY_PWD" #@param {type:"string"}
version = "latest" #@param ["latest", "Commit ID"] {allow-input: true}
check_GPU = False # @param {type:"boolean"}
use_cuda = False # @param {type:"boolean"}
use_supermerge = False # @param {type:"boolean"}

In [2]:
#@title (Optional) Configuration:: Model
use_model_chilloutmix = True # @param {type:"boolean"}
use_model_orange = True # @param {type:"boolean"}

In [3]:
#@title (Optional) Configuration: ControlNet
# @markdown ###ControlNet Reference
# @markdown  - https://github.com/lllyasviel/ControlNet
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
# @markdown  - https://huggingface.co/webui/ControlNet-modules-safetensors/

use_controlnet = False # @param {type:"boolean"}
# @markdown ---

# @markdown  After `use_controlnet` is ticked, select the feature you need
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = False # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}

In [4]:
#@title Check GPU
if check_GPU:
  !nvidia-smi

In [ ]:
#@title Install Painting Tool

%cd $root_dir

# install tool: stable-diffusion-webui
repo = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"
!git clone $repo

# time machine: commit or branch
if version:
  !git checkout $version

In [ ]:
#@title Install dependencies for Painting Tool

# install xformers
%cd {root_dir}
# !pip install xformers
!pip install xformers==0.0.17.dev464 # pre-release, fast calc
!python -m xformers.info

In [ ]:
#@title Install model
model_collections = []
# model_collections.append("https://huggingface.co/runwayml/stable-diffusion-v1-5")

vae_collections = []

if use_model_chilloutmix:
  model_collections.append("https://civitai.com/api/download/models/11732")
  
if use_model_orange:
  # model_collections.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3.safetensors")
  model_collections.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3.safetensors")
  vae_collections.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt")


%cd {tool_dir}/models/Stable-diffusion
for model in model_collections:
  !wget -nc --content-disposition {model}


%cd {tool_dir}/models/VAE
for vae in vae_collections:
  !wget -nc --content-disposition {vae}

In [11]:
#@title TODO: Tool config

In [12]:
#@title TODO: additional VAE

In [13]:
#@title TODO: embedding

In [ ]:
#@title Install extensions for Painting Tool

%cd {tool_dir}/extensions
!git clone https://github.com/bbc-mc/sdweb-merge-board
!git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
# !git clone 
# !git clone https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git
# !git clone https://github.com/tkalayci71/embedding-inspector
# !git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui
# !git clone https://github.com/Malisius/booru2prompt.git
# !git clone https://github.com/adieyal/sd-dynamic-prompts
# !git clone https://github.com/kousw/stable-diffusion-webui-daam
# !git clone https://github.com/Akegarasu/sd-webui-model-converter.git

In [15]:
#@title Install extension: supermerge

if use_supermerge:
  %cd {tool_dir}/extensions  
  !git clone https://github.com/hako-mikan/sd-webui-supermerger.git

In [16]:
#@title Install extension: controlnet

if use_controlnet:
  %cd {tool_dir}/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git

  control_net_models = []
  if control_canny == True:
      control_net_models.append("control_canny")
  if control_depth == True:
      control_net_models.append("control_depth")
  if control_hed == True:
      control_net_models.append("control_hed")
  if control_mlsd == True:
      control_net_models.append("control_mlsd")
  if control_normal == True:
      control_net_models.append("control_normal")
  if control_openpose == True:
      control_net_models.append("control_openpose")
  if control_scribble == True:
      control_net_models.append("control_scribble")
  if control_seg == True:
      control_net_models.append("control_seg")
  if t2iadapter_keypose == True:
      control_net_models.append("t2iadapter_keypose")
  if t2iadapter_seg == True:
      control_net_models.append("t2iadapter_seg")
  if t2iadapter_sketch == True:
      control_net_models.append("t2iadapter_sketch")

  if Path('sd-webui-controlnet').is_dir():
      %cd sd-webui-controlnet/models
      for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/{control_net_model}-fp16.safetensors

In [17]:
#@title Use CUDA
if use_cuda:
  # use cuda instead of cpu for large model (e.g., 7G)
  !sed -i 's/weight_load_location = None if cmd_opts.lowram else "cpu"/weight_load_location = None if cmd_opts.lowram else "cuda"/g' {tool_dir}/modules/shared.py

# Tips
Common Quality Prompt
> magnificent, best quality, masterpiece, ultra-detailed, an extremely delicate and beautiful, extremely detailed, Hyperdetail, global illumination, beautiful and delicate

Common Negative Prompt
> lowres, blurry, worst quality, low quality, normal quality,bad anatomy, disfigured, deformed, mutation, mutilated, ugly, totem pole,poorly drawn face,  cloned face, several faces, long neck, mutated hands, bad hands, poorly drawn hands,extra limbs, malformed limbs, missing arms, missing fingers, extra fingers, fused fingers, too many fingers,missing legs, extra legs, malformed legs, extra digit, fewer digits, glitchy, cropped, jpeg artifacts, signature, watermark, username, text, error

In [ ]:
#@title Launch
%cd {tool_dir}

# use param to run tool
# ref: https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings
# --no-progressbar-hiding (may slow down)
arg = f"--xformers --no-hashing --share --gradio-debug --gradio-auth {account}:{pwd} --disable-safe-unpickle --gradio-queue --enable-insecure-extension-access --no-half-vae"

!COMMANDLINE_ARGS="{arg}" REQS_FILE="requirements.txt" python launch.py